<a href="https://colab.research.google.com/github/Ekalabya3/Abhilash-2D2V/blob/main/Copy_of_Abhilash_2D_PIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [293]:
import numpy as np
from scipy.fft import fftfreq, fft2, ifft2
import matplotlib.pyplot as plt

In [294]:
def periodic_particles(x, y, length):

    rem = x % length;

    if x > length:
      x = rem;
    elif x < 0:
      x = length - rem;

    rem = y % length

    if y > length:
      y = rem;
    elif y < 0:
      y = length - rem ;


    return x, y

In [295]:
def ghost(rho):
    Nx, Ny = rho.shape
    for i in range(Nx):
        rho[i, 0] = rho[i, Ny - 2]
        rho[i, Ny - 1] = rho[i, 1]

    for j in range(Ny):
        rho[0, j] = rho[Nx - 2, j]
        rho[Nx - 1, j] = rho[1, j]
    return rho

In [296]:
def charge_density(pos_x, pos_y, dx, dy, length):    #DONE

    charge_density = np.zeros((int(ncell ** 0.5)+1 + 2  , int(ncell ** 0.5)+1 + 2 ))                                 #+ 2*nghost

    # LOOPING
    for p in range(nparticles):
        pos_x[p], pos_y[p] = periodic_particles(pos_x[p], pos_y[p], length)

        # GRID POINT FINDER
        i = int(pos_x[p] / dx)   +1
        j = int(pos_y[p] / dy)   +1

        # FRACTION FINDER
        x_frac = ((pos_x[p]+dx) - (i * dx)) / dx
        y_frac = ((pos_y[p]+dy)  - (j * dy)) / dy
        # WEIGHT FINDER
        weight_ii  = (1 - x_frac) * (1 - y_frac)    # FOR X=0, Y=0
        weight_ji  = x_frac * (1 - y_frac)          # FOR X=1, Y=0
        weight_ij  = (1 - x_frac) * y_frac          # FOR X=0, Y=1
        weight_jj  = x_frac * y_frac                # FOR X=1, Y=1

        charge_density[i][j]     =   charge_density[i][j]     + (particle_charge * weight_ii)           #/ (dx*dy)
        charge_density[(i+1)][j]   =   charge_density[(i+1)][j]   + (particle_charge * weight_ji)       #/ (dx*dy)
        charge_density[i][(j+1)]   =   charge_density[i][(j+1)]   + (particle_charge * weight_ij)       #/ (dx*dy)
        charge_density[(i+1)][(j+1)] =   charge_density[(i+1)][(j+1)] + (particle_charge * weight_jj)   #/ (dx*dy)

        #charge_density_without_ghost = charge_density[nghost :-nghost, nghost :-nghost]


        #transpose_charge_density = np.transpose(charge_density_without_ghost)
        #rho_1 = (np.flip(transpose_charge_density, 0)) / (dx*dy)

        rho_1 = charge_density / (dx * dy)

    return  rho_1

In [297]:
def calculate_background_charge_density(dx, dy, length, particle_charge, nparticles):  #DONE

    # FIRST: TOTAL CHARGE CALCULATION
    total_particle_charge = particle_charge * nparticles

    # SECOND: TOTAL NUMBER OF GRID POINTS
    num_x_grid_points = int(ncell ** 0.5) + 1
    num_y_grid_points = int(ncell ** 0.5) + 1
    total_grid_points = num_x_grid_points * num_y_grid_points

    # THIRD: CHARGE DENSITY BY BACKGROUND PARTICLES (IONS)
    background_charge_density = (np.ones((num_x_grid_points, num_y_grid_points)) * (-total_particle_charge / total_grid_points)) / (dx * dy)  #(length*length)
    # Add extra rows and columns filled with zeros around the matrix
    background_charge_density_frame = np.zeros((num_x_grid_points + 2, num_y_grid_points + 2))
    background_charge_density_frame [1:-1, 1:-1] = background_charge_density


    return background_charge_density, background_charge_density_frame

In [298]:
def poisson_solver(rho, dx, dy):
    # Calculate Wavenumbers
    k_x = fftfreq(rho.shape[1], dx)
    k_y = fftfreq(rho.shape[0], dy)

    # Compute Fourier transform of charge density
    rho_hat = fft2(rho)

    # Compute potential in Fourier space
    kx_grid, ky_grid = np.meshgrid(k_x, k_y, indexing='ij')
    kx_grid_sq = kx_grid ** 2
    ky_grid_sq = ky_grid ** 2
    k_sq = kx_grid_sq + ky_grid_sq

    # Avoid division by zero at zero frequency
    k_sq[0, 0] = 1.0
    potential_hat = rho_hat / (4 * np.pi ** 2 * k_sq)

    # Compute electric field components in Fourier space
    E_x_hat = -1j * 2 * np.pi * kx_grid * potential_hat
    E_y_hat = -1j * 2 * np.pi * ky_grid * potential_hat

    # Compute inverse Fourier transform to get electric field in real space
    E_x = np.real(ifft2(E_x_hat))
    E_y = np.real(ifft2(E_y_hat))
    electric_potential = np.real(ifft2(potential_hat))

    return E_x, E_y, k_sq, electric_potential

In [299]:
def interpolate_field (pos_x, pos_y, dx, dy, length, E_x, E_y):

    Ex_at_particle = np.zeros(len(pos_x))
    Ey_at_particle = np.zeros(len(pos_y))

    # LOOPING
    for p in range(nparticles):
        pos_x[p], pos_y[p] = periodic_particles(pos_x[p], pos_y[p], length)

        # GRID POINT FINDER
        i = (int(pos_x[p] / dx)) + 1
        j = (int(pos_y[p] / dy))  + 1

        # FRACTION FINDER
        x_frac = (pos_x[p] - (i * dx)) / dx
        y_frac = (pos_y[p] - (j * dy)) / dy

        # WEIGHT FINDER
        weight_ii  = (1 - x_frac) * (1 - y_frac)    # FOR X=0, Y=0
        weight_ji  = x_frac * (1 - y_frac)          # FOR X=1, Y=0
        weight_ij  = (1 - x_frac) * y_frac          # FOR X=0, Y=1
        weight_jj  = x_frac * y_frac                # FOR X=1, Y=1

        Ex_at_particle[p] = weight_ii * E_x[i][j] + weight_ji * E_x[(i+1)][j] + weight_ij * E_x[i][(j+1)] + weight_jj * E_x[(i+1)][(j+1)]
        Ey_at_particle[p] = weight_ii * E_y[i][j] + weight_ji * E_y[(i + 1)][j] + weight_ij * E_y[i][(j + 1)] + weight_jj * E_y[(i + 1)][(j + 1)]
        force_x = particle_charge * Ex_at_particle[p]
        force_y = particle_charge * Ey_at_particle[p]
        acc_x = force_x / particle_mass
        acc_y = force_y / particle_mass

    return Ex_at_particle, Ey_at_particle, force_x, force_y, acc_x, acc_y

In [300]:
def kinetic_energy(vel_x, vel_y, particle_mass):
    vel_sq = vel_x**2 + vel_y**2
    vel_resultant = np.sqrt(vel_sq)
    ke_particle = 0.5 * particle_mass * vel_resultant

    # Calculate total kinetic energy of the system
    ke_system = np.sum(ke_particle)

    return ke_particle, ke_system, vel_resultant

In [301]:
nparticles = 20    #input(print("ENTER THE NUMBER OF PARTICLES : "))
ncell = 9         #input(print("ENTER THE NUMBER OF CELLS [A PERFECT SQUARE] : "))
length = 10  # DO NOT CHANGE
#length = int(length)
n_step = 0#input(print("ENTER THE NUMBER OF TIMESTEPS : "))
t = 0
dt = 0.01 #input(print("ENTER SIZE OF TIMESTEP"))
particle_charge = 1
particle_mass = 1
nghost = 1
iteration = 0


In [302]:
position_x = np.random.uniform(0, length, nparticles)
position_y = np.random.uniform(0, length, nparticles)
#vel_x = np.random.uniform(-2, 2, nparticles)
#vel_y = np.random.uniform(-2, 2, nparticles)

In [303]:
dx = length / abs(ncell ** 0.5)
dy = length / abs(ncell ** 0.5)


In [304]:
'''
#pos_x_list = []
#vel_x_list = []
#rho_total_list = []
ke_system_list = []
timesteps_list = []
'''

'\n#pos_x_list = []\n#vel_x_list = []\n#rho_total_list = []\nke_system_list = []\ntimesteps_list = []\n'

In [305]:
' TESTS '

rho_electron = charge_density(position_x, position_y, dx, dy, length)
print("RHO_ELECTRON \n", rho_electron)
print("Total charge electron : ", np.sum(rho_electron))

background_charge_density, background_charge_density_frame = calculate_background_charge_density(dx, dy, length, particle_charge, nparticles)
print("RHO_Background \n", background_charge_density)
print("RHO_Background frame \n", background_charge_density_frame)
print("Total background charge : ", np.sum(background_charge_density))

rho_total = background_charge_density_frame + rho_electron
print("RHO total \n", rho_total)
print("Total charge system : ", np.sum(rho_total))

#print("bcd \n", background_charge_density)
#print("bcd with zeros\n",background_charge_density_with_zeros)

#print(" RHO: \n",rho_electron)
#print(np.sum(rho_total))
#print(force_x)
#rho = ghost(rho_total)
#print("RHO WITH out GHOST :\n", rho_total)

#print("RHO total with ghost :\n", rho)

RHO_ELECTRON 
 [[0.         0.         0.         0.         0.         0.        ]
 [0.         0.15934124 0.10310623 0.03361864 0.01480685 0.        ]
 [0.         0.16540791 0.13121259 0.07855494 0.12709597 0.        ]
 [0.         0.11886507 0.21134389 0.13787657 0.16509737 0.        ]
 [0.         0.11016067 0.18542405 0.05078299 0.00730503 0.        ]
 [0.         0.         0.         0.         0.         0.        ]]
Total charge electron :  1.7999999999999998
RHO_Background 
 [[-0.1125 -0.1125 -0.1125 -0.1125]
 [-0.1125 -0.1125 -0.1125 -0.1125]
 [-0.1125 -0.1125 -0.1125 -0.1125]
 [-0.1125 -0.1125 -0.1125 -0.1125]]
RHO_Background frame 
 [[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.     -0.1125 -0.1125 -0.1125 -0.1125  0.    ]
 [ 0.     -0.1125 -0.1125 -0.1125 -0.1125  0.    ]
 [ 0.     -0.1125 -0.1125 -0.1125 -0.1125  0.    ]
 [ 0.     -0.1125 -0.1125 -0.1125 -0.1125  0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]]
Total background charge :  -1.79999999

In [306]:

'''
for step in range(n_step):
    timesteps_list.append(iteration)
    #pos_x_list = position_x
    #print(" POSITION X COORDINATES FOR EACH ITERATIONS \n", pos_x_list)
    #vel_x_list.append(vel_x)
    ke_particle, ke_system, vel_resultant = kinetic_energy(vel_x, vel_y, particle_mass)
    ke_system_list.append(ke_system)


    rho_electron = charge_density(position_x, position_y, dx, dy, length)
    background_charge_density = calculate_background_charge_density(dx, dy, length, particle_charge, nparticles)
    rho_total = background_charge_density + rho_electron

    E_grid_x, E_grid_y, k_square, electric_potential_grid = poisson_solver(rho_total, dx, dy)

    Ex_at_particle, Ey_at_particle, force_x, force_y, acceleration_x, acceleration_y = interpolate_field(position_x, position_y, dx, dy, length, E_grid_x, E_grid_y)

    vx_half = vel_x + (0.5 * dt * acceleration_x)
    vy_half = vel_y + (0.5 * dt * acceleration_y)

    position_x += vx_half * dt
    position_y += vy_half * dt

    rho_electron = charge_density(position_x, position_y, dx, dy, length)
    background_charge_density = calculate_background_charge_density(dx, dy, length, particle_charge, nparticles)
    rho_total = background_charge_density + rho_electron

    E_grid_x, E_grid_y, k_square, electric_potential_grid = poisson_solver(rho_total, dx, dy)

    Ex_at_particle, Ey_at_particle, force_x, force_y, acceleration_x, acceleration_y = interpolate_field(position_x, position_y, dx, dy, length, E_grid_x, E_grid_y)

    vel_x = vx_half + 0.5 * dt * acceleration_x
    vel_y = vy_half + 0.5 * dt * acceleration_y


    #print("TOTAL CHARGE DENSITY OF SYSTEM : ", np.sum(rho_total))
    #print("TOTAL POTENTIAL OF SYSTEM : ", np.sum(electric_potential_grid))

    #rho_total_list.append(rho_total)
    iteration += 1
    print("ITERATION : ", iteration)

    # Plotting histogram
    #plt.hist(vel_resultant, bins=150, density=True, alpha=0.75)
    #plt.xlabel('Velocity')
    #plt.ylabel('Probability Density')
    #plt.title('Probability Distribution of Particle Velocities')
    #plt.grid(True)
    #plt.xlim([0, 1])  # Replace min_velocity and max_velocity with your desired limits
    #plt.ylim([0, 20])
    #plt.show()
    plt.imshow(rho_total, extent=(0, length, 0, length), origin='lower', cmap='viridis')
    plt.colorbar(label='Charge Density')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Charge Density Heatmap')
    plt.show()

    t += dt

'''

'\nfor step in range(n_step):\n    timesteps_list.append(iteration)\n    #pos_x_list = position_x\n    #print(" POSITION X COORDINATES FOR EACH ITERATIONS \n", pos_x_list)\n    #vel_x_list.append(vel_x)\n    ke_particle, ke_system, vel_resultant = kinetic_energy(vel_x, vel_y, particle_mass)\n    ke_system_list.append(ke_system)\n\n\n    rho_electron = charge_density(position_x, position_y, dx, dy, length)\n    background_charge_density = calculate_background_charge_density(dx, dy, length, particle_charge, nparticles)\n    rho_total = background_charge_density + rho_electron\n\n    E_grid_x, E_grid_y, k_square, electric_potential_grid = poisson_solver(rho_total, dx, dy)\n\n    Ex_at_particle, Ey_at_particle, force_x, force_y, acceleration_x, acceleration_y = interpolate_field(position_x, position_y, dx, dy, length, E_grid_x, E_grid_y)\n\n    vx_half = vel_x + (0.5 * dt * acceleration_x)\n    vy_half = vel_y + (0.5 * dt * acceleration_y)\n\n    position_x += vx_half * dt\n    position_y 

In [307]:
'''
#print("END POSITION X COORDINATES : \n ",pos_x_list)
print("Length of timesteps_list:", len(timesteps_list))
print("Length of ke_system_list:", len(ke_system_list))
'''

'\n#print("END POSITION X COORDINATES : \n ",pos_x_list)\nprint("Length of timesteps_list:", len(timesteps_list))\nprint("Length of ke_system_list:", len(ke_system_list))\n'

In [308]:
'''
plt.plot(timesteps_list, ke_system_list, linestyle='-')
plt.title('Kinetic Energy vs Timestep')
plt.xlabel('Timestep')
plt.ylabel('Kinetic Energy')
#plt.grid(True)
plt.show()




plt.imshow(rho_total, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='Charge Density')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Charge Density Heatmap')
plt.show()


plt.imshow(electric_potential_grid, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='ELECTRIC POTENTIAL')
plt.xlabel('x')
plt.ylabel('y')
plt.title('ELECTRIC POTENTIAL HEAT MAP')
plt.show()

plt.imshow(k_square, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='K SQUARE')
plt.xlabel('x')
plt.ylabel('y')
plt.title('K SQUARE HEAT MAP')
plt.show()




plt.imshow(rho_electron, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='Charge Density')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Charge Density Heatmap')
plt.show()

plt.imshow(background_charge_density, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='Charge Density')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Charge Density Heatmap')
plt.show()

plt.imshow(rho_total, extent=(0, length, 0, length), origin='lower', cmap='viridis')
plt.colorbar(label='Charge Density')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Charge Density Heatmap')
plt.show()
'''

"\nplt.plot(timesteps_list, ke_system_list, linestyle='-')\nplt.title('Kinetic Energy vs Timestep')\nplt.xlabel('Timestep')\nplt.ylabel('Kinetic Energy')\n#plt.grid(True)\nplt.show()\n\n\n\n\nplt.imshow(rho_total, extent=(0, length, 0, length), origin='lower', cmap='viridis')\nplt.colorbar(label='Charge Density')\nplt.xlabel('x')\nplt.ylabel('y')\nplt.title('Charge Density Heatmap')\nplt.show()\n\n\nplt.imshow(electric_potential_grid, extent=(0, length, 0, length), origin='lower', cmap='viridis')\nplt.colorbar(label='ELECTRIC POTENTIAL')\nplt.xlabel('x')\nplt.ylabel('y')\nplt.title('ELECTRIC POTENTIAL HEAT MAP')\nplt.show()\n\nplt.imshow(k_square, extent=(0, length, 0, length), origin='lower', cmap='viridis')\nplt.colorbar(label='K SQUARE')\nplt.xlabel('x')\nplt.ylabel('y')\nplt.title('K SQUARE HEAT MAP')\nplt.show()\n\n\n\n\nplt.imshow(rho_electron, extent=(0, length, 0, length), origin='lower', cmap='viridis')\nplt.colorbar(label='Charge Density')\nplt.xlabel('x')\nplt.ylabel('y')\np